In [45]:
#watch -n 1 nvidia-smi
# 터미널에 실행해서 gpu 보면서 실행시키기

In [2]:
import sys, os, distutils.core

In [3]:
# !python -m pip install pyyaml==5.1
# # Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# # See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
# !git clone 'https://github.com/facebookresearch/detectron2' 

In [4]:
print(sys.path)

['/opt/conda/lib/python311.zip', '/opt/conda/lib/python3.11', '/opt/conda/lib/python3.11/lib-dynload', '', '/opt/conda/lib/python3.11/site-packages', '/home/jovyan/snukdt/AdelaiDet', '/opt/conda/lib/python3.11/site-packages/pydot-3.0.3-py3.11.egg', '/opt/conda/lib/python3.11/site-packages/future-1.0.0-py3.11.egg', '/opt/conda/lib/python3.11/site-packages/setuptools-75.6.0-py3.11.egg', '/home/jovyan/snukdt/BCNet', '/opt/conda/lib/python3.11/site-packages/Pillow-6.2.2-py3.11-linux-x86_64.egg', '/home/jovyan/snukdt', '/home/jovyan/snukdt/detectron2', '/opt/conda/lib/python3.11/site-packages/setuptools/_vendor']


In [5]:
import random
import torch
import math
import copy
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import logging

from detectron2 import model_zoo
from detectron2.config import LazyConfig, instantiate, get_cfg
from detectron2.data import (
    MetadataCatalog,
    DatasetCatalog,
    build_detection_train_loader,
    build_detection_test_loader
)
from detectron2.engine import SimpleTrainer, DefaultPredictor, hooks, DefaultTrainer, HookBase
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, DatasetEvaluators
from detectron2.utils.logger import setup_logger
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import CommonMetricPrinter, JSONWriter, TensorboardXWriter, EventStorage
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.structures import BoxMode
import detectron2.data
from detectron2.data import detection_utils as utils


from adet.config import get_cfg as get_adet_cfg  # AdelaiDet 전용 get_cfg

import matplotlib.pyplot as plt

# 로깅 설정
setup_logger()
logging.getLogger('matplotlib.font_manager').setLevel(logging.WARNING)
logging.getLogger("PIL").setLevel(logging.WARNING)

In [6]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/bin/bash: line 1: nvcc: command not found
torch:  2.4 ; cuda:  cu118
detectron2: 0.6


In [7]:
# 현재 경로를 Python 경로에 추가
adet_path = "/home/jovyan/snukdt/fcos_model/AdelaiDet"
if adet_path not in sys.path:
    sys.path.insert(0, adet_path)

In [8]:
# Set GPU device
torch.cuda.set_device(0)

In [9]:
def set_seed(seed=42):
    """Seed 설정 함수"""
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def apply_gaussian_noise(image, mean=0, std=25):
    """
    이미지에 가우시안 노이즈를 추가하는 함수.

    Args:
        image (np.ndarray): 원본 이미지 (H x W x C)
        mean (float): 노이즈의 평균값
        std (float): 노이즈의 표준 편차

    Returns:
        np.ndarray: 노이즈가 추가된 이미지
    """
    # 정규 분포에서 노이즈 생성
    noise = np.random.normal(mean, std, image.shape).astype(np.float32)
    # 원본 이미지에 노이즈 추가
    noisy_image = image.astype(np.float32) + noise
    # 픽셀 값 범위를 0~255로 클리핑
    return np.clip(noisy_image, 0, 255).astype(np.uint8)


In [38]:
import os
import json
import numpy as np
import random
import torch
import cv2
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from detectron2.data import build_detection_train_loader, build_detection_test_loader
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator
from detectron2 import model_zoo


# ---------------------------------------------------------------------------- #
# 1) get_custom_dicts
# ---------------------------------------------------------------------------- #
def get_custom_dicts(img_dir, json_file):
    """
    Detectron2 형식의 데이터셋을 생성. 클래스가 1개로 고정된 데이터셋 처리.

    Args:
        img_dir (str): 이미지 파일이 위치한 디렉토리 경로
        json_file (str): COCO 형식 JSON 파일 경로

    Returns:
        list[dict]: Detectron2 형식의 데이터셋
    """
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 이미지 및 어노테이션 매핑
    images = {img["id"]: img for img in data["images"]}
    annotations = {}

    # 어노테이션 필터링 및 처리
    for ann in data["annotations"]:
        image_id = ann["image_id"]
        ann_copy = ann.copy()

        # 단일 클래스이므로 모든 category_id를 0으로 설정
        ann_copy["category_id"] = 0
        annotations.setdefault(image_id, []).append(ann_copy)

    # Detectron2 형식 데이터셋 생성
    dataset_dicts = []
    for image_id, img_info in images.items():
        filtered_annotations = [
            {
                "bbox": ann["bbox"],
                "bbox_mode": BoxMode.XYWH_ABS,
                "category_id": ann["category_id"]
            }
            for ann in annotations.get(image_id, [])
        ]

        # 어노테이션이 없는 이미지는 제외
        if filtered_annotations:
            record = {
                "file_name": os.path.join(img_dir, img_info["file_name"]),
                "image_id": image_id,
                "height": img_info["height"],
                "width": img_info["width"],
                "annotations": filtered_annotations,
            }
            dataset_dicts.append(record)

    return dataset_dicts
    
# ---------------------------------------------------------------------------- #
# 2) 데이터셋 Split
# ---------------------------------------------------------------------------- #
def split_dataset(dataset, train_size=2000, val_size=318, test_size=319, seed=42):
    """
    데이터를 훈련, 검증, 테스트 세트로 나눕니다.
    
    Args:
        dataset (list): 전체 데이터셋
        train_size (int): 훈련 데이터 개수
        val_size (int): 검증 데이터 개수
        test_size (int): 테스트 데이터 개수
        seed (int): 무작위성을 고정하기 위한 시드
        
    Returns:
        tuple: (train, val, test) 데이터셋
    """
    random.seed(seed)
    random.shuffle(dataset)
    return (
        dataset[:train_size], 
        dataset[train_size:train_size + val_size], 
        dataset[train_size + val_size:]
    )


# ---------------------------------------------------------------------------- #
# 3) 커스텀 Mapper (320x320 Resize)
# ---------------------------------------------------------------------------- #
from detectron2.data.transforms import AugInput

def custom_mapper(dataset_dict):
    """
    Detectron2 데이터셋 매퍼 함수, 이미지와 어노테이션을 증강 처리.

    Args:
        dataset_dict (dict): 데이터셋 정보 딕셔너리

    Returns:
        dict: 변환된 데이터셋
    """
    dataset_dict = dataset_dict.copy()
    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    # Augmentation 설정
    augmentation = T.AugmentationList([
        T.Resize((320, 320)),               # Resize
        T.RandomFlip(prob=0.5),            # 랜덤 플립
        T.RandomBrightness(0.8, 1.2),      # 밝기 조정
        T.RandomContrast(0.8, 1.2),        # 대비 조정
        T.RandomSaturation(0.8, 1.2),      # 채도 조정
    ])

    # AugInput 객체 생성
    aug_input = AugInput(image)
    transform = augmentation(aug_input)  # Transform 적용
    image_augmented = aug_input.image    # 변환된 이미지 가져오기

    # 가우시안 노이즈 추가 (50% 확률로 적용)
    if np.random.rand() < 0.5:
        image_augmented = apply_gaussian_noise(image_augmented, mean=0, std=25)

    # 바운딩 박스 변환
    if "annotations" in dataset_dict:
        for anno in dataset_dict["annotations"]:
            x1, y1, w, h = anno["bbox"]
            x2, y2 = x1 + w, y1 + h
            bbox = np.array([[x1, y1, x2, y2]], dtype="float32")
            transformed_bbox = transform.apply_box(bbox)[0]
            anno["bbox"] = [
                transformed_bbox[0],
                transformed_bbox[1],
                transformed_bbox[2] - transformed_bbox[0],
                transformed_bbox[3] - transformed_bbox[1],
            ]

    # 이미지를 텐서로 변환
    dataset_dict["image"] = torch.as_tensor(image_augmented.transpose(2, 0, 1).astype("float32"))
    dataset_dict["height"], dataset_dict["width"] = 320, 320

    # Detectron2 인스턴스 생성
    if "annotations" in dataset_dict:
        annos = [
            utils.transform_instance_annotations(
                obj, transforms=[], image_size=(320, 320)
            )
            for obj in dataset_dict["annotations"]
        ]
        dataset_dict["instances"] = utils.annotations_to_instances(
            annos, (320, 320)
        )

    return dataset_dict



# ---------------------------------------------------------------------------- #
# 4) DataLoader 빌더
# ---------------------------------------------------------------------------- #
def build_train_loader(cfg):
    return build_detection_train_loader(cfg, mapper=custom_mapper)

# def build_test_loader(cfg):
#     return build_detection_test_loader(cfg, dataset_name=cfg.DATASETS.TEST[0], mapper=custom_mapper)


# ---------------------------------------------------------------------------- #
# 5) CustomTrainer
# ---------------------------------------------------------------------------- #
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_train_loader(cfg)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, cfg, True, output_dir=cfg.OUTPUT_DIR)



In [39]:
print(os.getcwd())


# 삭제할 인덱스 정의
delete_indices = set()

# 크기 0인 바운딩박스 포함하는 이미지 삭제
delete_indices.update([92, 152, 261, 383, 444, 447, 576, 715, 898, 974, 1049, 1050, 1149, 1354, 1355, 1378])
# width 40 넘는 것 삭제
delete_indices.update([233,  276, 1062, 1112, 1140])

print(len(delete_indices))


/home/jovyan/snukdt
21


In [40]:
# ---------------------------------------------------------------------------- #
# 6) 데이터셋 등록 및 Split
# ---------------------------------------------------------------------------- #
img_dir = "./cropped_output/image"
json_file = "./cropped_output/json/cropped_annotations_xywh.json"


# 클래스 이름 및 색상 설정 (Hiddenlens only)
class_names = ["Hiddenlens"]


# 데이터셋 준비
all_data = get_custom_dicts(img_dir, json_file)

# 1. delete_indices에 해당하는 이미지 제거
all_data = [d for d in all_data if d["image_id"] not in delete_indices]


train_size = 1000
val_size = 184
test_size = len(all_data) - (train_size + val_size)

train_data, val_data, test_data = split_dataset(all_data, train_size, val_size, seed=42)


print(f"Train={len(train_data)}, Val={len(val_data)}, Test={len(test_data)}")

# DatasetCatalog 등록 (기존 이름이 있으면 제거)
for d in ["train", "val", "test"]:
    name = f"cropped_{d}"
    if name in DatasetCatalog.list():
        DatasetCatalog.remove(name)

# Detectron2에 등록
DatasetCatalog.register("cropped_train", lambda: train_data)
DatasetCatalog.register("cropped_val", lambda: val_data)
DatasetCatalog.register("cropped_test", lambda: test_data)

MetadataCatalog.get("cropped_train").set(thing_classes=["object"])
MetadataCatalog.get("cropped_val").set(thing_classes=["object"])
MetadataCatalog.get("cropped_test").set(thing_classes=["object"])



Train=1000, Val=184, Test=184


namespace(name='cropped_test', thing_classes=['object'])

#### 데이터 분포 확인

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 바운딩박스 크기 데이터 추출
bbox_data = []
for data in all_data:
    for annotation in data["annotations"]:
        x, y, w, h = annotation["bbox"]
        bbox_data.append({"image_id": data["image_id"], "bbox_width": w, "bbox_height": h})

# DataFrame으로 변환
df_bboxes = pd.DataFrame(bbox_data)

# 데이터 분포 통계 확인
print("=== Bounding Box Data ===")
print(df_bboxes[["bbox_width", "bbox_height"]].describe())

# 데이터 분포 시각화
plt.figure(figsize=(9, 6))
plt.hist(df_bboxes["bbox_width"], bins=20, alpha=0.7, label="BBox Width")
plt.hist(df_bboxes["bbox_height"], bins=20, alpha=0.7, label="BBox Height")
plt.xlabel("Pixels")
plt.ylabel("Frequency")
plt.title("Bounding Box Size Distribution")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 바운딩박스 크기 데이터 추출 및 리사이즈 적용
bbox_data_resized = []
for data in all_data:
    orig_width, orig_height = data["width"], data["height"]
    scale_x = 320 / orig_width
    scale_y = 320 / orig_height

    for annotation in data["annotations"]:
        x, y, w, h = annotation["bbox"]
        resized_w = w * scale_x
        resized_h = h * scale_y
        bbox_data_resized.append({"image_id": data["image_id"], "resized_bbox_width": resized_w, "resized_bbox_height": resized_h})

# DataFrame으로 변환
df_bboxes_resized = pd.DataFrame(bbox_data_resized)

# 데이터 분포 통계 확인
print("=== Resized Bounding Box Data ===")
print(df_bboxes_resized[["resized_bbox_width", "resized_bbox_height"]].describe())

# 데이터 분포 시각화
plt.figure(figsize=(9, 6))
plt.hist(df_bboxes_resized["resized_bbox_width"], bins=20, alpha=0.7, label="Resized BBox Width")
plt.hist(df_bboxes_resized["resized_bbox_height"], bins=20, alpha=0.7, label="Resized BBox Height")
plt.xlabel("Pixels")
plt.ylabel("Frequency")
plt.title("Resized Bounding Box Size Distribution (320x320)")
plt.legend()
plt.show()

#### 이미지 변환 출력

In [25]:
import os
print(os.path.exists("./cropped_output/IMG_04392.png"))

True


In [ ]:
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt

img = cv2.imread("./cropped_output/KakaoTalk_Photo_2025-01-21-20-56-41.png")
noisy_img = apply_gaussian_noise(img, mean=0, std=100)

# 원본 이미지
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(img[:, :, ::-1])  # BGR -> RGB
plt.title("Original Image")
plt.axis("off")

# 가우시안 노이즈 적용 이미지
plt.subplot(1, 2, 2)
plt.imshow(noisy_img[:, :, ::-1])  # BGR -> RGB
plt.title("Gaussian Noise Applied")
plt.axis("off")

plt.show()

In [ ]:
from detectron2.utils.visualizer import Visualizer
import matplotlib.pyplot as plt

for d in random.sample(DatasetCatalog.get("cropped_train"), 5):
    img = cv2.imread(d["file_name"])
    noisy_img = apply_gaussian_noise(img, mean=0, std=25)

    # 원본 이미지
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img[:, :, ::-1])  # BGR -> RGB
    plt.title("Original Image")
    plt.axis("off")

    # 가우시안 노이즈 적용 이미지
    plt.subplot(1, 2, 2)
    plt.imshow(noisy_img[:, :, ::-1])  # BGR -> RGB
    plt.title("Gaussian Noise Applied")
    plt.axis("off")

    plt.show()

In [ ]:
# 시각화
import matplotlib.pyplot as plt
import cv2
import random
from detectron2.utils.visualizer import Visualizer

# 필요한 경우 다른 폰트로 변경할 수 있습니다.
plt.rcParams["font.family"] = "DejaVu Sans"



sample_data = DatasetCatalog.get("cropped_train")
for d in random.sample(sample_data, 3):
    print("File:", d["file_name"])
    img = cv2.imread(d["file_name"])  # 기존 방식
    # img = imread_unicode(d["file_name"])  # 바꾼 방식
    if img is None:
        print("Can't read:", d["file_name"])
        continue
    visualizer = Visualizer(img, MetadataCatalog.get("cropped_train"), scale=0.8)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.axis("off")
    plt.show()

## TRAIN

#### final

In [41]:

# ---------------------------------------------------------------------------- #
# 7) 훈련 실행
# ---------------------------------------------------------------------------- #
set_seed(seed = 42)

output_dir = "./Crop_OutPut_Folder/finaltest_cropped_output"

# AdelaiDet 전용 설정 로드
cfg = get_adet_cfg()
cfg.merge_from_file("./AdelaiDet/configs/FCOS-Detection/FCOS_RT/MS_DLA_34_4x_syncbn.yaml")
cfg.DATALOADER.NUM_WORKERS = 2
# P2단 추가 설정
cfg.MODEL.FPN.IN_FEATURES = ["level2", "level3", "level4", "level5"] 
cfg.MODEL.FCOS.IN_FEATURES = ["p2"]  # P2~P5 사용
cfg.MODEL.FCOS.FPN_STRIDES = [4]             # P2=4로 설정
cfg.MODEL.FCOS.SIZES_OF_INTEREST = [40]        # P2단의 interest=40으로 설정

# 이미지 크기 설정(Min=800, Max=1333)
cfg.INPUT.MIN_SIZE_TEST = 300
cfg.INPUT.MAX_SIZE_TEST = 400
    
cfg.MODEL.FCOS.NUM_CLASSES = 1  # 모델 클래스 수 설정
cfg.DATASETS.TRAIN = ("cropped_train",)
cfg.DATASETS.TEST = ("cropped_val",)


# 학습 기간 연장
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.MAX_ITER = 18000  # 더 많은 iteration
cfg.TEST.EVAL_PERIOD = 9000
cfg.OUTPUT_DIR = output_dir  # 학습 결과 저장 디렉토리
cfg.SOLVER.BASE_LR = 0.00025  # 학습률 설정
cfg.MODEL.FCOS.FOCAL_LOSS_GAMMA = 4  # 기본값: 2
# cfg.SOLVER.STEPS = [30000]
# cfg.SOLVER.GAMMA = 0.1  # 학습률 감소 비율

cfg.MODEL.DEVICE = "cuda"
cfg.MODEL.WEIGHTS = "./OutPut_Folder/output_nut_new_train_10000/model_final.pth"  # 사전 학습 가중치 설정

# 출력 디렉토리 생성
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[01/22 01:36:01 d2.engine.defaults]: Model:
OneStageDetector(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bottom_up): DLA(
      (base_layer): Sequential(
        (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
        (1): SyncBatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
     

/opt/conda/lib/python3.11/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/22 01:36:13 d2.utils.events]:  eta: 2:45:55  iter: 19  total_loss: 2.129  loss_fcos_cls: 1.052  loss_fcos_loc: 0.422  loss_fcos_ctr: 0.6584    time: 0.5591  last_time: 0.5761  data_time: 0.3337  last_data_time: 0.3341   lr: 4.9953e-06  max_mem: 4178M


2025-01-22 01:36:13.771198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-22 01:36:13.775836: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-22 01:36:13.781064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737509773.788811 1111155 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737509773.791841 1111155 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

[01/22 01:36:55 d2.utils.events]:  eta: 2:38:45  iter: 39  total_loss: 1.938  loss_fcos_cls: 0.8614  loss_fcos_loc: 0.4202  loss_fcos_ctr: 0.674    time: 0.5229  last_time: 0.6421  data_time: 0.2909  last_data_time: 0.3340   lr: 9.9902e-06  max_mem: 4178M
[01/22 01:37:05 d2.utils.events]:  eta: 2:37:32  iter: 59  total_loss: 1.661  loss_fcos_cls: 0.5451  loss_fcos_loc: 0.4026  loss_fcos_ctr: 0.6598    time: 0.5256  last_time: 0.4295  data_time: 0.3391  last_data_time: 0.2628   lr: 1.4985e-05  max_mem: 4178M
[01/22 01:37:16 d2.utils.events]:  eta: 2:36:56  iter: 79  total_loss: 1.492  loss_fcos_cls: 0.4168  loss_fcos_loc: 0.393  loss_fcos_ctr: 0.6486    time: 0.5281  last_time: 0.6800  data_time: 0.3132  last_data_time: 0.4328   lr: 1.998e-05  max_mem: 4178M
[01/22 01:37:27 d2.utils.events]:  eta: 2:35:43  iter: 99  total_loss: 1.379  loss_fcos_cls: 0.3528  loss_fcos_loc: 0.3642  loss_fcos_ctr: 0.6496    time: 0.5270  last_time: 0.8973  data_time: 0.2922  last_data_time: 0.6859   lr: 2.

## Test

In [42]:
print(output_dir)

./Crop_OutPut_Folder/finaltest_cropped_output


In [43]:
# 필요한 모듈 Import
from detectron2.engine import DefaultPredictor  # 탐지 모델의 기본 Predictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset  # COCO 평가와 데이터셋 추론 함수
from detectron2.data import build_detection_test_loader  # 테스트 데이터 로더 생성 함수
from pycocotools.coco import COCO  # COCO 데이터셋 처리를 위한 모듈
from pycocotools.cocoeval import COCOeval  # COCO 평가 수행을 위한 모듈
import numpy as np  # 배열 연산 라이브러리

# 출력 디렉토리 설정
outdir = "./Performance_Folder/{}".format(output_dir.split('/')[2])  # output_dir의 하위 폴더 경로 생성

# 모델 설정
cfg.MODEL.WEIGHTS = '{}/model_final.pth'.format(output_dir)  # 훈련 완료된 모델 가중치 파일 경로 설정
predictor = DefaultPredictor(cfg)  # DefaultPredictor로 모델 로드

# 테스트셋 평가 준비
evaluator = COCOEvaluator("cropped_test", cfg, False, output_dir=outdir)  # COCO 평가 객체 생성
# Custom Mapper를 적용한 테스트 데이터 로더 생성
test_loader = build_detection_test_loader(cfg, "cropped_test")  # 테스트 데이터 로더에 커스텀 매퍼 적용

# 탐지 결과 생성 및 평가 수행
print(inference_on_dataset(predictor.model, test_loader, evaluator))  # 테스트셋에 대한 평가 결과 출력 (2열 결과)

[01/22 04:09:56 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./Crop_OutPut_Folder/finaltest_cropped_output/model_final.pth ...
WARNING [01/22 04:09:56 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[01/22 04:09:56 d2.evaluation.coco_evaluation]: Trying to convert 'cropped_test' to COCO format ...
WARNING [01/22 04:09:56 d2.data.datasets.coco]: Using previously cached COCO format annotations at './Performance_Folder/finaltest_cropped_output/cropped_test_coco_format.json'. You need to clear the cache file if your dataset has been modified.
[01/22 04:09:56 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   object   | 185          |
|            |              |
[01/22 04:09:56 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_lengt

/opt/conda/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

[01/22 04:09:57 d2.evaluation.evaluator]: Inference done 11/184. Dataloading: 0.0008 s/iter. Inference: 0.0232 s/iter. Eval: 0.0003 s/iter. Total: 0.0243 s/iter. ETA=0:00:04
[01/22 04:10:02 d2.evaluation.evaluator]: Total inference time: 0:00:05.061069 (0.028274 s / iter per device, on 1 devices)
[01/22 04:10:02 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:02 (0.016631 s / iter per device, on 1 devices)
[01/22 04:10:02 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[01/22 04:10:02 d2.evaluation.coco_evaluation]: Saving results to ./Performance_Folder/finaltest_cropped_output/coco_instances_results.json
[01/22 04:10:02 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
[01/22 04:10:02 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[01/22 04:10:02 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.01 s

In [44]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np

# outdir = output_dir
# "./Crop_OutPut_Folder/{}".format(output_dir.split('/')[2])
gt_file = "./Performance_Folder/cropped_output_transferlearning/cropped_test_coco_format.json"   
# gt_file = "./Performance_Folder/lens_8000_P2P3P4_640/custom_test_coco_format.json"# GT 파일 경로
dt_file = f"{outdir}/coco_instances_results.json"     # Detection 결과 파일 경로
print(gt_file)
coco_gt = COCO(gt_file)
coco_dt = coco_gt.loadRes(dt_file)

coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
# 평가 실행: 모든 이미지에 대해 매칭, IoU 계산 등 내부 평가 수행
coco_eval.evaluate()

# 평가 누적: 모든 IoU, 카테고리, 영역 범위, maxDet 조건을 기반으로 결과 집계
coco_eval.accumulate()

# 1. 기본 summarize 호출로 전체 AP, AR 등 기본 평가 지표(표준 COCO metrics) 출력
print("COCO Evaluator 결과 출력")
# coco_eval.summarize()
print("")

recall = coco_eval.eval['recall']
iou_thrs = 0.5 + np.arange(10)*0.05

cat_ids = coco_gt.getCatIds()
cats = coco_gt.loadCats(cat_ids)
cat_names = [c['name'] for c in cats]
print(cat_names)

if recall.size > 0:
    desired_ious = [0.5, 0.6, 0.7, 0.8, 0.9]
    max_det_idx = 2
    
    # Hiddenlens 클래스에 대한 area=all AR 계산
    if "object" in cat_names:
        hiddenlens_idx = cat_names.index("object")

        # IoU=0.5~0.9, area=all
        area_all_idx = 0
        for iou_target in desired_ious:
            iou_idx_candidates = np.where(np.isclose(iou_thrs, iou_target, atol=1e-6))[0]
            if len(iou_idx_candidates) == 0:
                print(f"IoU={iou_target} is not in iou_thrs")
                continue
            iou_idx = iou_idx_candidates[0]

            cat_vals_hiddenlens = recall[iou_idx, hiddenlens_idx, area_all_idx, max_det_idx]
            cat_valid_hiddenlens = cat_vals_hiddenlens[cat_vals_hiddenlens >= 0]

            if cat_valid_hiddenlens.size == 0:
                print(f"Category: Hiddenlens - AR (IoU={iou_target}, area=all): No valid data")
            else:
                cat_ar_hiddenlens = np.mean(cat_valid_hiddenlens)
                print(f"Category: Hiddenlens - AR (IoU={iou_target}, area=all): {cat_ar_hiddenlens:.3f}")

        print("")

        # 먼저 IoU=0.5, area=small일 때 전체 카테고리에 대한 AR 출력
        area_small_idx = 1
        iou_05_index_candidates = np.where(np.isclose(iou_thrs, 0.5, atol=1e-6))[0]
        if len(iou_05_index_candidates) == 0:
            print("IoU=0.5 is not in iou_thrs")
        else:
            iou_05_index = iou_05_index_candidates[0]
            vals_small = recall[iou_05_index, :, area_small_idx, max_det_idx]
            valid_small_vals = vals_small[vals_small >= 0]
            if valid_small_vals.size == 0:
                print("\nNo valid data for IoU=0.5 | area=small | maxDets=100")
            else:
                ar_small_iou_0_5 = np.mean(valid_small_vals)
                print(f"\nAverage Recall (AR) @ IoU=0.5 | area=small | maxDets=100: {ar_small_iou_0_5:.3f}")
                print('')

        # IoU=0.5~0.9, area=small
        for iou_target in desired_ious:
            iou_idx_candidates = np.where(np.isclose(iou_thrs, iou_target, atol=1e-6))[0]
            if len(iou_idx_candidates) == 0:
                print(f"IoU={iou_target} is not in iou_thrs")
                continue
            iou_idx = iou_idx_candidates[0]

            cat_vals_hiddenlens = recall[iou_idx, hiddenlens_idx, area_small_idx, max_det_idx]
            cat_valid_hiddenlens = cat_vals_hiddenlens[cat_vals_hiddenlens >= 0]

            if cat_valid_hiddenlens.size == 0:
                print(f"Category: Hiddenlens - AR (IoU={iou_target}, area=small): No valid data")
            else:
                cat_ar_hiddenlens = np.mean(cat_valid_hiddenlens)
                print(f"Category: Hiddenlens - AR (IoU={iou_target}, area=small): {cat_ar_hiddenlens:.3f}")
    else:
        print("No 'Hiddenlens' category found in annotations.")


./Performance_Folder/cropped_output_transferlearning/cropped_test_coco_format.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
COCO Evaluator 결과 출력

['object']
Category: Hiddenlens - AR (IoU=0.5, area=all): 0.951
Category: Hiddenlens - AR (IoU=0.6, area=all): 0.827
Category: Hiddenlens - AR (IoU=0.7, area=all): 0.681
Category: Hiddenlens - AR (IoU=0.8, area=all): 0.422
Category: Hiddenlens - AR (IoU=0.9, area=all): 0.059


Average Recall (AR) @ IoU=0.5 | area=small | maxDets=100: 0.937

Category: Hiddenlens - AR (IoU=0.5, area=small): 0.937
Category: Hiddenlens - AR (IoU=0.6, area=small): 0.793
Category: Hiddenlens - AR (IoU=0.7, area=small): 0.577
Category: Hiddenlens - AR (IoU=0.8, area=small): 0.297
Category: Hiddenlens - AR (IoU=0.9, a

### testing

In [31]:
import cv2
import numpy as np
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog
import torch

def visualize_prediction(predictor, dataset_dict):
    """
    이미지에 대한 예측 결과와 Ground Truth를 시각화하여 비교
    """
    # 이미지 읽기
    img = cv2.imread(dataset_dict["file_name"])
    
    # 예측 수행
    outputs = predictor(img)
    
    # 예측 결과 출력
    print("\n=== Prediction Results ===")
    print(f"Number of detected boxes: {len(outputs['instances'])}")
    if len(outputs['instances']) > 0:
        print("Predicted boxes (x1, y1, x2, y2):")
        print(outputs['instances'].pred_boxes.tensor.cpu().numpy())
        print("Confidence scores:", outputs['instances'].scores.cpu().numpy())

    # Ground Truth 시각화
    v1 = Visualizer(img[:, :, ::-1],
                   metadata=MetadataCatalog.get("nuts_test"),
                   scale=1.0)
    gt_vis = v1.draw_dataset_dict(dataset_dict)
    gt_image = gt_vis.get_image()[:, :, ::-1]

    # 예측 결과 시각화
    v2 = Visualizer(img[:, :, ::-1],
                   metadata=MetadataCatalog.get("nuts_test"),
                   scale=1.0)
    pred_vis = v2.draw_instance_predictions(outputs["instances"].to("cpu"))
    pred_image = pred_vis.get_image()[:, :, ::-1]

    # 이미지 저장
    cv2.imwrite('ground_truth.jpg', gt_image)
    cv2.imwrite('prediction.jpg', pred_image)
    
    print("\n=== Ground Truth ===")
    print(f"Number of GT boxes: {len(dataset_dict['annotations'])}")
    print("GT boxes (x, y, w, h):")
    for anno in dataset_dict['annotations']:
        print(anno['bbox'])

    return gt_image, pred_image

def compare_predictions():
    # 테스트 데이터셋 가져오기
    test_dataset = DatasetCatalog.get("nuts_test")
    
    # Predictor 생성
    predictor = DefaultPredictor(cfg)
    
    # 처음 5개 이미지에 대해 예측 수행
    for i in range(min(5, len(test_dataset))):
        print(f"\nProcessing image {i+1}")
        gt_img, pred_img = visualize_prediction(predictor, test_dataset[i])
        
        # IOU 계산 (선택적)
        if len(test_dataset[i]['annotations']) > 0:
            outputs = predictor(cv2.imread(test_dataset[i]["file_name"]))
            if len(outputs['instances']) > 0:
                pred_boxes = outputs['instances'].pred_boxes.tensor.cpu().numpy()
                gt_boxes = np.array([
                    [anno['bbox'][0], anno['bbox'][1],
                     anno['bbox'][0] + anno['bbox'][2],
                     anno['bbox'][1] + anno['bbox'][3]]
                    for anno in test_dataset[i]['annotations']
                ])
                print("\nIOU Scores:")
                for pred_box in pred_boxes:
                    ious = []
                    for gt_box in gt_boxes:
                        # IOU 계산
                        x1 = max(pred_box[0], gt_box[0])
                        y1 = max(pred_box[1], gt_box[1])
                        x2 = min(pred_box[2], gt_box[2])
                        y2 = min(pred_box[3], gt_box[3])
                        
                        intersection = max(0, x2 - x1) * max(0, y2 - y1)
                        pred_area = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])
                        gt_area = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
                        union = pred_area + gt_area - intersection

                        
                        iou = intersection / union if union > 0 else 0
                        ious.append(iou)
                    print(f"Pred box IoU scores: {ious}")

# 실행
compare_predictions()

[01/17 02:19:14 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./OutPut_Folder/output_nut_singleclass_10000/model_final.pth ...

Processing image 1


/opt/conda/lib/python3.11/site-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))


=== Prediction Results ===
Number of detected boxes: 3
Predicted boxes (x1, y1, x2, y2):
[[1443.1904   925.6636  1562.0431   994.85693]
 [1400.6007  1087.716   1526.7201  1166.5267 ]
 [1587.9829  1042.3646  1716.6158  1125.0533 ]]
Confidence scores: [0.81502956 0.7794371  0.7503651 ]


/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44204 (\N{HANGUL SYLLABLE GYEON}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44284 (\N{HANGUL SYLLABLE GWA}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 47448 (\N{HANGUL SYLLABLE RYU}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44204 (\N{HANGUL SYLLABLE GYEON}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44284 (\N{HANGUL SYLLABLE GWA}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/


=== Ground Truth ===
Number of GT boxes: 3
GT boxes (x, y, w, h):
[1399.8859784698484, 1088.6175474181935, 128.113808656027, 79.64970163207522]
[1588.3296032391218, 1042.6830736515597, 128.82191633146545, 82.6751640066525]
[1445.7601283850104, 928.6944771064989, 115.66491278499188, 66.38160384735318]

IOU Scores:
Pred box IoU scores: [0.0, 0.0, 0.9276911586924345]
Pred box IoU scores: [0.9523085074067592, 0.0, 0.0]
Pred box IoU scores: [0.0, 0.9857348164889523, 0.0]

Processing image 2

=== Prediction Results ===
Number of detected boxes: 3
Predicted boxes (x1, y1, x2, y2):
[[1562.9891   937.33356 1654.913   1052.5996 ]
 [1469.9827   605.4349  1696.6294   843.56323]
 [1565.8405   540.2022  1747.323    640.8593 ]]
Confidence scores: [0.2989753  0.21344036 0.1500677 ]


...


=== Ground Truth ===
Number of GT boxes: 2
GT boxes (x, y, w, h):
[1618.1506796302144, 912.7679756268495, 68.16981211226584, 73.98274182726539]
[1683.6782509629352, 1057.0343221900168, 57.600848994085254, 57.600848994085254]

IOU Scores:
Pred box IoU scores: [0.7957469344500702, 0.0]
Pred box IoU scores: [0.0, 0.7418119039967855]
Pred box IoU scores: [0.0, 0.0]

Processing image 4

=== Prediction Results ===
Number of detected boxes: 2
Predicted boxes (x1, y1, x2, y2):
[[1539.4988   918.0034  1704.5365   996.5937 ]
 [ 659.051    108.32382  849.56757  276.53635]]
Confidence scores: [0.8806498  0.22454992]


/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44204 (\N{HANGUL SYLLABLE GYEON}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44284 (\N{HANGUL SYLLABLE GWA}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 47448 (\N{HANGUL SYLLABLE RYU}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44204 (\N{HANGUL SYLLABLE GYEON}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44284 (\N{HANGUL SYLLABLE GWA}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/


=== Ground Truth ===
Number of GT boxes: 1
GT boxes (x, y, w, h):
[1537.140872914537, 916.7028218063848, 170.90752798318522, 85.63993092213877]

IOU Scores:
Pred box IoU scores: [0.8861648274180446]
Pred box IoU scores: [0.0]

Processing image 5

=== Prediction Results ===
Number of detected boxes: 5
Predicted boxes (x1, y1, x2, y2):
[[2067.0112  1364.0435  2162.459   1455.0348 ]
 [2136.0173   898.37897 2243.3914   954.5769 ]
 [1703.9958  1136.7144  1811.4375  1196.2986 ]
 [1465.6522   970.7956  1560.1743  1041.7052 ]
 [1997.5111   835.9263  2069.8975   921.4558 ]]
Confidence scores: [0.8664936  0.82843214 0.7655959  0.76532465 0.68712115]


/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44204 (\N{HANGUL SYLLABLE GYEON}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44284 (\N{HANGUL SYLLABLE GWA}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 47448 (\N{HANGUL SYLLABLE RYU}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44204 (\N{HANGUL SYLLABLE GYEON}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/detectron2/utils/visualizer.py:318: UserWarning: Glyph 44284 (\N{HANGUL SYLLABLE GWA}) missing from current font.
  s, (width, height) = canvas.print_to_buffer()
/home/jovyan/snukdt/detectron2/


=== Ground Truth ===
Number of GT boxes: 5
GT boxes (x, y, w, h):
[1463.4562505493595, 973.8790457812079, 92.67528101159223, 68.6887365620737]
[1703.8668449451302, 1141.784847421598, 109.57488276010258, 55.605149373022414]
[2001.3940515372117, 843.2721096899771, 67.02258617320126, 80.05472513787788]
[2144.547469343867, 904.8363620386701, 99.08800265755099, 53.54445004077343]
[2066.8317666210623, 1369.566271412582, 98.94395753397248, 85.09185284963542]

IOU Scores:
Pred box IoU scores: [0.0, 0.0, 0.0, 0.0, 0.9041916847817825]
Pred box IoU scores: [0.0, 0.0, 0.0, 0.7654285346790551, 0.0]
Pred box IoU scores: [0.0, 0.8823895945586953, 0.0, 0.0, 0.0]
Pred box IoU scores: [0.8853703307365942, 0.0, 0.0, 0.0, 0.0]
Pred box IoU scores: [0.0, 0.0, 0.8295755358593295, 0.0, 0.0]
